In [83]:
# import the necessary libs
import pandas
import numpy
import os
import zipfile

import matplotlib.pyplot as plt 
import h5py
import types 

import librosa # we will make use of `features` and `filters` modules
import hmmlearn # make use of hmm

import sklearn.linear_model
import sklearn.pipeline
import sklearn.preprocessing
import sklearn.metrics

import functools
import itertools
import multiprocessing

import dask_jobqueue
import dask.distributed

In [84]:
rpm_lower = 715
rpm_uppper = 915

win_len_ms = 1000 # sample length (ms)

skip = 5000 # the number of measurements to skip from the start of each recording
smapling_freq = 4096 # sampling rate (Hz)

In [85]:
# the recordings to be used during training
n_good,n_bad = '0D', '3D'
# define the file path of the datasets
infile = '/home/ix502iv/Documents/Probabilistic_Graphical_Models/DataSet/dataset_hmm.zip'

In [86]:
def load_zipfile(zfile, n):
    win_len = int(win_len_ms/1000*smapling_freq)
    with zfile.open(n + '.csv', 'r') as f:
        data = pandas.read_csv(f).iloc[skip:, :]

    n = (data.shape[0]//win_len) * win_len
    data = data.iloc[:n, :]

    rpm = numpy.reshape(data['Measured_RPM'].values, (-1, win_len), order='C')
    vibr = numpy.reshape(data['Vibration_3'].values, (-1, win_len), order='C')
    #choosing rpm based on the sensitivity aspect of hmm_mfcc : rpm_lw < rpm < rpm_up.
    ind, = numpy.nonzero(numpy.all(rpm>rpm_lower, axis=1) & numpy.all(rpm<rpm_uppper, axis=1))
    # randomly permutate a sequence : return a permutated range
    numpy.random.seed(170287); ind = numpy.random.permutation(ind)
    return vibr[ind, :].copy()

In [87]:
def load_data(filename, n_good, n_bad):
    with zipfile.ZipFile(filename, 'r') as zfile:
        good = load_zipfile(zfile, n_good) # 0D
        bad = load_zipfile(zfile, n_bad) # 3D
    return good, bad

In [88]:
# lets load the files
good, bad = load_data(infile, n_good, n_bad)

In [91]:
print("Good Samples '#0D'", good.shape)

Good Samples '#0D' (709, 4096)


In [92]:
print("Bad Samples '3D'", bad.shape)

Bad Samples '3D' (708, 4096)
